In [19]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from ast import literal_eval

In [12]:
data = pd.read_csv('./drain/results/normal.csv', engine='c', na_filter=False, memory_map=True)
blockId_list= data['BlockId'].tolist()
seqs = data['EventSequence'].apply(literal_eval).tolist()

In [13]:
data.head()

,BlockId,EventSequence
0,blk_-1608999687919862906,"[1, 2, 1, 1, 3, 3, 4, 4, 3, 4, 5, 5, 5, 6, 1, ..."
1,blk_7503483334202473044,"[1, 1, 2, 1, 3, 4, 3, 4, 3, 4, 5, 5, 5, 10]"
2,blk_-9073992586687739851,"[1, 2, 1, 1, 3, 4, 3, 4, 3, 4, 5, 5, 5]"
3,blk_7854771516489510256,"[1, 1, 2, 1]"
4,blk_1717858812220360316,"[1, 1, 2, 1]"


In [20]:
num_sessions = 0
window_size = 10
inputs=[]
outputs = []
name = 'test'


loading data: 100%|███████████████████████████████████████████████████████████████| 382/382 [00:00<00:00, 42450.89it/s]

Number of sessions(test): 382
Number of seqs(test): 2143


In [23]:
name = './drain/results/normal.csv'
generate_train_data(name)

loading data: 100%|███████████████████████████████████████████████████████████████| 382/382 [00:00<00:00, 38242.89it/s]

Number of sessions(./drain/results/normal.csv): 382
Number of seqs(./drain/results/normal.csv): 2143


In [22]:
def generate_train_data(name):
    num_sessions = 0
    inputs = []
    outputs = []
    data = pd.read_csv(name, engine='c', na_filter=False, memory_map=True)
    blockId_list= data['BlockId'].tolist()
    seqs = data['EventSequence'].apply(literal_eval).tolist()
    for line in tqdm(seqs, "loading data"):
        num_sessions += 1
        seq = [0] + line + [30] + [31] * (window_size - 1)
        line = tuple(seq)

        for i in range(len(line) - window_size):
            inputs.append(line[i:i + window_size])
            outputs.append(line[i + window_size])
    print('Number of sessions({}): {}'.format(name, num_sessions))
    print('Number of seqs({}): {}'.format(name, len(inputs)))
    dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
    return dataset

In [ ]:
def generate_test_data(name,window_size=10):
    hdfs = set()
    # hdfs = []
    with open(name, 'r') as f:
        for ln in f.readlines():
            ln = [0]+list(map(lambda n: n, map(int, ln.strip().split())))+[30]
            ln = ln + [-1] * (window_size + 1 - len(ln))
            hdfs.add(tuple(ln))
            # hdfs.append(tuple(ln))
    session_to_seq = []
    seqs = []
    labels = []
    seq_count = 0
    for line in tqdm(hdfs, "normal:"):
        session = []
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            label = line[i + window_size]
            seqs.append(seq)
            session.append(seq_count)
            labels.append(label)
            seq_count += 1
        session_to_seq.append(session)
    print('Number of sessions({}): {}'.format(name, len(session_to_seq)))
    print('Number of seqs({}): {}'.format(name, len(seqs)))
    dataset = TensorDataset(torch.tensor(seqs, dtype=torch.float), torch.tensor(labels))

    return session_to_seq, dataset, seqs,labels